<div align="center">
    <a href="https://github.com/syubogdanov/hse-howto-python">
        <img src="https://cdn-icons-png.flaticon.com/128/1864/1864551.png" height="128px" width="auto">
    </a>
    <h3>
        <b>
            Продвинутый Python
        </b>
    </h3>
    <i>
        Абстрактные синтаксические деревья
    </i>
</div>

<br>

**Цель занятия.** Изучение базовых инструментов для работы с абстрактными синтаксическими деревьями программ, написанных на языке программирования Python.

**Определение.** Абстрактное синтаксическое дерево - это представление программы в виде дерева, состоящего из ее синтаксических единиц.

**Пример.** Абстрактное синтаксическое дерево для некоторой программы.

In [7]:
code: str = \
"""
var: int = 1 + 2 + 3
"""

In [8]:
import ast

tree: ast.Module = ast.parse(code)

In [9]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        AnnAssign(
            target=Name(id='var', ctx=Store()),
            annotation=Name(id='int', ctx=Load()),
            value=BinOp(
                left=BinOp(
                    left=Constant(value=1),
                    op=Add(),
                    right=Constant(value=2)),
                op=Add(),
                right=Constant(value=3)),
            simple=1)],
    type_ignores=[])


**Пояснение.** В атрибуте `body` построенного дерева хранится список, состоящий из последовательно идущих синтаксических единиц программы. В частности, для примера выше единственной единицей является `ast.AnnAssign` - присвоение значения переменной, для которой указали тип данных. Далее, в узле можно увидеть более специфичную информацию:

- `target` - какой переменной было присвоено значение;
- `annotation` - какую аннотацию назначили переменной;
- `value` - значение, которое присвоили переменной;
- `simple` - показывает, отсутствует ли переменная, которой присваивают значение, в правой части.

**Пример.** Абстрактное синтаксическое дерево для некоторой программы.

In [19]:
code: str = \
"""
a: int = 10
b: float = -5.0

var = a + b
"""

In [20]:
tree: ast.Module = ast.parse(code)

In [21]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        AnnAssign(
            target=Name(id='a', ctx=Store()),
            annotation=Name(id='int', ctx=Load()),
            value=Constant(value=10),
            simple=1),
        AnnAssign(
            target=Name(id='b', ctx=Store()),
            annotation=Name(id='float', ctx=Load()),
            value=UnaryOp(
                op=USub(),
                operand=Constant(value=5.0)),
            simple=1),
        Assign(
            targets=[
                Name(id='var', ctx=Store())],
            value=BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Name(id='b', ctx=Load())))],
    type_ignores=[])


**Пояснение.** В программе выше находятся три смысловые синтаксические единицы:

- Присвоение для `a`;
- Присовение для `b`;
- Присвоение для `var`.

По этой причине тело модуля, которому отвечает узел `ast.Module`, содержит в атрибуте `body` три последовательно идущих узла, соответствующих операциям, указанным выше. Обратите внимание, что присвоение с аннотацией и присовение без нее - это два разных объекта.

**Пример.** Абстрактное синтаксическое дерево для некоторой программы.

In [22]:
code: str = \
"""
for iteration in range(10):
    print("Hello, world!")
"""

In [23]:
tree: ast.Module = ast.parse(code)

In [24]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        For(
            target=Name(id='iteration', ctx=Store()),
            iter=Call(
                func=Name(id='range', ctx=Load()),
                args=[
                    Constant(value=10)],
                keywords=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Constant(value='Hello, world!')],
                        keywords=[]))],
            orelse=[])],
    type_ignores=[])


**Примечание.** Цикл `for` отлично иллюстрирует древовидную структуру программы.

**Упражнение.** Вспомните, как можно использовать `else` вместе с циклами. Напишите такую программу. Изучите ее абстрактное синтаксическое дерево.

**Пример.** Абстрактное синтаксическое дерево для некоторой программы.

In [25]:
code: str = \
"""
mydict = {
    1: 2,
    3: 4,
}

for key, value in mydict.items():
    print(f"{key}: {value}")
"""

In [26]:
tree: ast.Module = ast.parse(code)

In [27]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        Assign(
            targets=[
                Name(id='mydict', ctx=Store())],
            value=Dict(
                keys=[
                    Constant(value=1),
                    Constant(value=3)],
                values=[
                    Constant(value=2),
                    Constant(value=4)])),
        For(
            target=Tuple(
                elts=[
                    Name(id='key', ctx=Store()),
                    Name(id='value', ctx=Store())],
                ctx=Store()),
            iter=Call(
                func=Attribute(
                    value=Name(id='mydict', ctx=Load()),
                    attr='items',
                    ctx=Load()),
                args=[],
                keywords=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            JoinedStr(
                                v

**Пояснение.** Пример выше иллюстрирует достаточно комплексную программу.

Давайте рассмотрим инициализацию словаря. Как и в примерах выше, это все так же `ast.Assign`, однако со структурной точки зрения более интересно представление словаря в виде узла. Обратите внимание на хранение ключей и значений - это два отдельных списка. Один - для ключей, второй - для значений.

Далее, рассмотрим цикл `for`. Первое - перебираемые аргументы. Их несколько, поэтому узел для перебираемых значений (атрибут `target`) передставляет собой кортеж из переменных. Итерируемым объектом (атрибут `iter`) выступает вызов (узел `ast.Call`) атрибута (узел `ast.Attribute`) под названием `items` у переменной `mydict`.

Не менее интересным является представление `f`-строки - узел `FormattedValue`, в атрибуте `values` которого содержатся все значения, требующие вывода на стандартный поток. Узел `FormattedValue` представляет объект, который выводится при помощи фигурных скобок.

**Упражнение.** Попробуйте использовать другой форматированный вывод - через `%`. Изменится ли абстрактное синтаксическое дерево? Если да, то проинтерпретируйте полученный результат.

**Упражнение.** Напишите некоторую программу в одну строку при помощи `;`. Как выглядит абстрактное синтаксическое дерево?

**Рассуждение.** Одним из преимуществ работы с абстрактными синтаксическими деревьями является возможность преобразований кода. Например, можно упрощать какие-то узлы программы. Возможности ограничены только Вашей фантазией.

**Пример.** Изменение имени переменной через абстрактное синтаксческое дерево.

In [9]:
code: str = \
"""
var: float = 2.7
"""

In [10]:
tree: ast.Module = ast.parse(code)

In [11]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        AnnAssign(
            target=Name(id='var', ctx=Store()),
            annotation=Name(id='float', ctx=Load()),
            value=Constant(value=2.7),
            simple=1)],
    type_ignores=[])


In [12]:
node: ast.AnnAssign = tree.body[0]
variable: ast.Name = node.target

variable.id: str = "exponent"

In [13]:
code: str = ast.unparse(tree)
print(code)

exponent: float = 2.7


**Пояснение.** Зная, как выглядит абстрактное синтаксическое дерево для примера выше, можно по атрибутам дойти до нужного узла и заменить в нем атрибут, ответственный за имя переменной. Далее, пользуемся функцией `ast.unparse`, которая по дереву строит программу, и получаем модифицированную программу.

**Замечание.** При обратной трансформации модуль `ast` не следит за тем, чтобы полученный код было возможно скомпилировать. Например, Вы можете использовать ключевое слово в качестве имени переменной - и Вам это сойдет с рук.

**Пример.** Изменение имени переменной на ключевое слово через абстрактное синтаксческое дерево.

In [22]:
code: str = \
"""
var: float = 2.7
"""

In [23]:
tree: ast.Module = ast.parse(code)

In [24]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        AnnAssign(
            target=Name(id='var', ctx=Store()),
            annotation=Name(id='float', ctx=Load()),
            value=Constant(value=2.7),
            simple=1)],
    type_ignores=[])


In [17]:
node: ast.AnnAssign = tree.body[0]
variable: ast.Name = node.target

variable.id: str = "while"

In [18]:
code: str = ast.unparse(tree)
print(code)

while: float = 2.7


**Упражнение.** Попробуйте вернуть код к прежнему состоянию. Получится ли у Вас это сделать?

**Замечание.** Код, который будет подвержен преобразованию в абстрактное синтаксическое дерево, должен обязательно быть валидным с точки зрения возможности компиляции программы.

**Пример.** Сортировка импортов через абстрактные синтаксические деревья.

In [29]:
code: str = \
"""
import numpy as np
import ast

a: int = 0
print("I love Python | a =", a)

import requests
"""

In [30]:
tree: ast.Module = ast.parse(code)

In [31]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        Import(
            names=[
                alias(name='numpy', asname='np')]),
        Import(
            names=[
                alias(name='ast')]),
        AnnAssign(
            target=Name(id='a', ctx=Store()),
            annotation=Name(id='int', ctx=Load()),
            value=Constant(value=0),
            simple=1),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Constant(value='I love Python | a ='),
                    Name(id='a', ctx=Load())],
                keywords=[])),
        Import(
            names=[
                alias(name='requests')])],
    type_ignores=[])


In [32]:
from typing import Any


imports: list[ast.Import] = []
other: list[Any] = []

for node in tree.body:

    if isinstance(node, ast.Import):
        imports.append(node)
    else:
        other.append(node)

imports.sort(key=lambda node: node.names[0].name)
tree.body = imports + other

In [33]:
code: str = ast.unparse(tree)
print(code)

import ast
import numpy as np
import requests
a: int = 0
print('I love Python | a =', a)


**Замечание.** Импорты вида `import ...` и импорты вида `from ... import ...` имеют различное узловое представление.

**Замечание.** Обратите внимание, что в примерах выше программы, с которыми была произведена работа, были строго линейными: никаких операторов ветвления, функций и так далее. Такая особенность позволяла работать только с атрибутом `body` у `ast.Module`. В общем случае, когда программа имеет достаточно большое число ветвлений, работа с узлами сильно усложняется. Давайте рассмотрим пример.

**Пример.** Подстановка значения `EXPONENT`, равного 2.7, вместо переменных через абстрактные синтаксические деревья.

In [42]:
code: str = \
"""
a: float = 5.0 * EXPONENT
print("5.0 * EXPONENT =", a)

print("EXPONENT =", EXPONENT)
"""

In [43]:
tree: ast.Module = ast.parse(code)

In [44]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        AnnAssign(
            target=Name(id='a', ctx=Store()),
            annotation=Name(id='float', ctx=Load()),
            value=BinOp(
                left=Constant(value=5.0),
                op=Mult(),
                right=Name(id='EXPONENT', ctx=Load())),
            simple=1),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Constant(value='5.0 * EXPONENT ='),
                    Name(id='a', ctx=Load())],
                keywords=[])),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Constant(value='EXPONENT ='),
                    Name(id='EXPONENT', ctx=Load())],
                keywords=[]))],
    type_ignores=[])


**Замечание.** Обратите внимание, что для решения задачи достаточно заменить все узлы типа `ast.Name`, такие что их атрибут `id` равен `EXPONENT`, на узел типа `ast.Constant`, в котором хранится значение искомой переменной.

**Пояснение.** Для такой маленькой и, на первый взгляд, несложной задачи, уже возникают серьезные проблемы. Чтобы раскрыть значения переменной, необходимо в каждом из возможных узлов дойти до максимального глубоко уровня и проверить, не используется ли там константа `EXPONENT`. Разработчики библиотеки `ast` об этом позаботились и реализовали класс, предназначенный для перебора всех узлов.

In [45]:
from __future__ import annotations


class Expander(ast.NodeTransformer):
    __slots__: tuple[str, ...] = ()

    def visit_Name(self: Expander, node: ast.Name) -> ast.Constant | ast.Name:
        if node.id != "EXPONENT":
            return node
        
        return ast.Constant(value=2.7)

In [46]:
expander = Expander()
expander.visit(tree)

In [47]:
code: str = ast.unparse(tree)
print(code)

a: float = 5.0 * 2.7
print('5.0 * EXPONENT =', a)
print('EXPONENT =', 2.7)


**Пояснение.** Для трансформаций абстрактных синтаксических деревьев принято использовать наследников класса `ast.NodeTransformer`. В дочернем классе разработчик должен переопределить методы вида `visit_Node`, где `Node` - тип узла, который нужно изменить. В частности, для примера выше был переопределен метод `visit_Name`. Далее, внутри функции-обработчика задать поведение по следующему принципу:

1. Если возвращаемое из функции значение равно `None`, тогда синтаксическая единица будет удалена из дерева;
2. Если возвращаемое из функции значение не равно `None`, тогда в абстрактное синтаксическое дерево, вместо текущего узла, будет подставлено возвращаемое значение.

В примере выше каждый узел, который относился к типу `ast.Name`, был трансформирован при помощи метода `visit_Name` в соответствии со следующим правилом: если имя переменной равно `EXPONENT`, то подставляем на место узла константу (узел `ast.Constant`), а в противном случае - оставляем узел нетронутым.

**Замечание.** Если наследник `ast.NodeTransformer` добавляет в дерево новые узлы (которые не были частью исходного дерева), тогда после преобразований необходимо дополнительно вызывать функцию `fix_missing_locations`. Она пересчитает информацию о расположении узлов, чтобы позже дерево могло безошибочно трансформироваться в код.

In [48]:
tree: ast.Module = ast.parse(code)

expander = Expander()
expander.visit(tree)

tree: ast.Module = ast.fix_missing_locations(tree)

In [49]:
code: str = ast.unparse(tree)
print(code)

a: float = 5.0 * 2.7
print('5.0 * EXPONENT =', a)
print('EXPONENT =', 2.7)


**Пример.** Вынести все импорты модулей в начало программы.

In [113]:
code: str = \
"""
import numpy as np

def function():
    import pandas as pd
    pass

if True:
    import ast
    pass
"""

In [114]:
tree: ast.Module = ast.parse(code)

In [115]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        Import(
            names=[
                alias(name='numpy', asname='np')]),
        FunctionDef(
            name='function',
            args=arguments(
                posonlyargs=[],
                args=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Import(
                    names=[
                        alias(name='pandas', asname='pd')]),
                Pass()],
            decorator_list=[]),
        If(
            test=Constant(value=True),
            body=[
                Import(
                    names=[
                        alias(name='ast')]),
                Pass()],
            orelse=[])],
    type_ignores=[])


In [116]:
class Collector(ast.NodeTransformer):
    __slots__: tuple[str] = ("imports")

    def __init__(self: Collector) -> None:
        super().__init__()
        self.imports: list[ast.Import, ...] = []

    def visit_Import(self: Collector, node: ast.Import) -> None:
        self.imports.append(node)
        return node


class Remover(ast.NodeTransformer):
    __slots__: tuple[str, ...] = ()

    def visit_Import(self: Remover, node: ast.Import) -> None:
        return None

In [117]:
collector = Collector()
collector.visit(tree)

tree = ast.fix_missing_locations(tree)

In [118]:
remover = Remover()
remover.visit(tree)

tree = ast.fix_missing_locations(tree)

In [119]:
tree.body = collector.imports + tree.body
tree = ast.fix_missing_locations(tree)

In [120]:
code: str = ast.unparse(tree)
print(code)

import numpy as np
import pandas as pd
import ast

def function():
    pass
if True:
    pass


**Пояснение.** Созданные классы реализуют следующий функционал:

- Класс `Collector` обходит дерево и собирает в нем импорты в поле `imports`;
- Класс `Remover` обходит дерево и удаляет из него импорты;

Как итог, остается запустить поочередно `Collector` и `Remover`, а затем перенести импорты из атрибута `imports` в начало абстрактного синтаксического дерева. Важно не забывать применять функцию `ast.fix_missing_locations`, чтобы всегда оставалась возможность преобразования дерева в код.

**Замечание.** В общем случае рекомендуется использовать наследников `ast.NodeTransformer` в следующем формате: `tree = YourTransformer().visit(tree)`. Так Вы не будете создавать дополнительную нагрузку на оперативную память после обработки дерева. В частности, обратите внимание на класс `Collector` - при текущей постановке задачи нужно не забывать очищать за ним память, поскольку в его атрибуте `imports` хранится список из ссылок на импорты.

**Пример.** В произвольной программе замените во всех строках слово `X` на слово `Y`. Например, если `X` равен `HSE`, а `Y` равен `Python`, то тогда строка `"I love HSE"` будет заменена на `"I love Python"`. Имена переменных, содержащие слово `X`, не должны быть заменены.

In [134]:
code = \
"""
HSE = 'Hello, do not change this variable'
text = 'Hey! It is HSE right above me!'                  # (!)

print(HSE)
print(text)

if True:
    text += "Sir, but what about HSE?"                   # (!)

for _ in range(1):
    for __ in range(2):
        for ___ in range(3):
            HSE += ">HSE<"                               # (!)

if 1 + 1 < 0:
    print(HSE)
else:
    for _ in range(50):
        print(HSE)


for badword in {"HSE", "Study at HSE", "Secret: HSE"}:   # (!)
    HSE = "HSE"                                          # (!)
"""

In [135]:
tree: ast.Module = ast.parse(code)

In [136]:
class Corrector(ast.NodeTransformer):
    __slots__: tuple[str, ...] = (
        "__src",
        "__dst",
    )

    def __init__(self: Collector, src: str, dst: str) -> None:
        super().__init__()

        self.__src: str = src
        self.__dst: str = dst

    def visit_Constant(self: Collector, node: ast.Constant) -> ast.Constant:
        if not isinstance(node.value, str):
            return node

        string: str = node.value.replace(
            self.__src,
            self.__dst,
        )

        return ast.Constant(value=string)

In [137]:
tree = Corrector(src="HSE", dst="Python").visit(tree)
tree = ast.fix_missing_locations(tree)

In [138]:
code: str = ast.unparse(tree)
print(code)

HSE = 'Hello, do not change this variable'
text = 'Hey! It is Python right above me!'
print(HSE)
print(text)
if True:
    text += 'Sir, but what about Python?'
for _ in range(1):
    for __ in range(2):
        for ___ in range(3):
            HSE += '>Python<'
if 1 + 1 < 0:
    print(HSE)
else:
    for _ in range(50):
        print(HSE)
for badword in {'Python', 'Study at Python', 'Secret: Python'}:
    HSE = 'Python'


**Замечание.** Обратите внимание на то, что модуль `ast` удаляет из кода комментарии, выполненные при помощи `#`. Учитывайте эту особенность, если ожидаете писать линтер на основе абстрактных синтаксических деревьев.

**Пример.** В произвольной функции удалите внутри каждого `if`-блока вызов функции `print`. Предполагайте, что код останется валидным.

In [139]:
code = \
"""
if True:
    a = 1
    print("1")

    if True:
        b = 2
        print("2")

    if True:
        c = 3
        print("3")

if True and True:
    d = 4
    print("4")

    if not False:
        e = 5
        print("5")
"""

In [140]:
tree: ast.Module = ast.parse(code)

In [141]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        If(
            test=Constant(value=True),
            body=[
                Assign(
                    targets=[
                        Name(id='a', ctx=Store())],
                    value=Constant(value=1)),
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Constant(value='1')],
                        keywords=[])),
                If(
                    test=Constant(value=True),
                    body=[
                        Assign(
                            targets=[
                                Name(id='b', ctx=Store())],
                            value=Constant(value=2)),
                        Expr(
                            value=Call(
                                func=Name(id='print', ctx=Load()),
                                args=[
                                    Constant(value='2')],
              

In [142]:
class Remover(ast.NodeTransformer):
    __slots__: tuple[str, ...] = ()

    def visit_If(self: Remover, node: ast.If) -> ast.If:
        body: list[Any] = []
        for child in node.body:

            if not isinstance(child, ast.Expr):
                body.append(child)
                continue

            if not isinstance(child.value, ast.Call):
                body.append(child)
                continue

            if not isinstance(child.value.func, ast.Name):
                body.append(child)
                continue

            if child.value.func.id != "print":
                body.append(child)
                continue

        node.body = body
        return node

In [143]:
tree = Remover().visit(tree)
tree = ast.fix_missing_locations(tree)

In [144]:
code: str = ast.unparse(tree)
print(code)

if True:
    a = 1
    if True:
        b = 2
        print('2')
    if True:
        c = 3
        print('3')
if True and True:
    d = 4
    if not False:
        e = 5
        print('5')


**Замечание.** По какой-то причине были удалены только те функции `print`, что находятся близко к корню абстрактного синтаксического дерева. Так происходит из-за внутреннего устройства наследников `ast.NodeTranformer`. Алгоритм, перебирающий дерево, останавливается на первом подходящем узле из текущей группы узлов. Именно по этой причине были удалены только те `print`, которые лежат на поверхности. Проблему решает использование метода `generic_visit`.

In [146]:
code = \
"""
if True:
    a = 1
    print("1")

    if True:
        b = 2
        print("2")

    if True:
        c = 3
        print("3")

if True and True:
    d = 4
    print("4")

    if not False:
        e = 5
        print("5")
"""

In [147]:
class Remover(ast.NodeTransformer):
    __slots__: tuple[str, ...] = ()

    def visit_If(self: Remover, node: ast.If) -> ast.If:
        self.generic_visit(node)

        body: list[Any] = []
        for child in node.body:

            if not isinstance(child, ast.Expr):
                body.append(child)
                continue

            if not isinstance(child.value, ast.Call):
                body.append(child)
                continue

            if not isinstance(child.value.func, ast.Name):
                body.append(child)
                continue

            if child.value.func.id != "print":
                body.append(child)
                continue

        node.body = body
        return node

In [148]:
tree = Remover().visit(tree)
tree = ast.fix_missing_locations(tree)

In [149]:
code: str = ast.unparse(tree)
print(code)

if True:
    a = 1
    if True:
        b = 2
    if True:
        c = 3
if True and True:
    d = 4
    if not False:
        e = 5


**Пояснение.** Метод `generic_visit` вызывает обработку для дочерних узлов текущего узла, если таковые имеются. В случаях, когда Вы не используете этот метод, дочерние узлы становятся Вашей зоной ответственности. Методы типа `visit` не будут изучать дочерние узлы того же типа, что и родитель, без `generic_visit`. Пример выше иллюстрирует этот кейс.

**Пример.** Отсортируйте все функции (включая вложенные) по имени.

In [173]:
code = \
"""
def b():
    pass

def a():
    pass

class A(object):
    def c():
        pass

    def b():
        def b():
            pass

        def a():
            pass

    def a():
        pass
"""

In [174]:
tree: ast.Module = ast.parse(code)

In [175]:
repr: str = ast.dump(tree, indent=4)
print(repr)

Module(
    body=[
        FunctionDef(
            name='b',
            args=arguments(
                posonlyargs=[],
                args=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Pass()],
            decorator_list=[]),
        FunctionDef(
            name='a',
            args=arguments(
                posonlyargs=[],
                args=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Pass()],
            decorator_list=[]),
        ClassDef(
            name='A',
            bases=[
                Name(id='object', ctx=Load())],
            keywords=[],
            body=[
                FunctionDef(
                    name='c',
                    args=arguments(
                        posonlyargs=[],
                        args=[],
                        kwonlyargs=[],
                        kw_d

In [176]:
from typing import TypeAlias, Union

Scope: TypeAlias = Union[ast.FunctionDef, ast.ClassDef, ast.Module]


class Sorter(ast.NodeTransformer):
    __slots__: tuple[str, ...] = ()

    def sort_scope(self: Sorter, node: Scope) -> Scope:
        functions: list[ast.FunctionDef] = []
        other: list[Any] = []

        for childnode in node.body:
            if isinstance(childnode, ast.FunctionDef):
                functions.append(childnode)
            else:
                other.append(childnode)

        functions.sort(key=lambda foo: foo.name)
        node.body = functions + other

        return ast.fix_missing_locations(node)

    def visit_FunctionDef(self: Sorter, node: ast.FunctionDef) -> ast.FunctionDef:
        self.generic_visit(node)
        return self.sort_scope(node)

    def visit_ClassDef(self: Sorter, node: ast.ClassDef) -> ast.ClassDef:
        self.generic_visit(node)
        return self.sort_scope(node)

    def visit_Module(self: Sorter, node: ast.Module) -> ast.Module:
        self.generic_visit(node)
        return self.sort_scope(node)

In [177]:
tree = Sorter().visit(tree)
tree = ast.fix_missing_locations(tree)

In [178]:
code: str = ast.unparse(tree)
print(code)

def a():
    pass

def b():
    pass

class A(object):

    def a():
        pass

    def b():

        def a():
            pass

        def b():
            pass

    def c():
        pass


**Пояснение.** Достаточно внутри каждого пространства имен, которое может содержать функции, выполнить следующие шаги:

1. Обработать дочерние узлы;
2. Обработать сам узел - отсортировать функции по имени.

**Пример.** Получение документации функции через абстрактные синтаксические деревья.

In [179]:
code = \
"""
def function(param: Any) -> Any:
    '''
    Args:
        param: Literally anything!

    Return:
        Literally anything!
    '''
"""

In [180]:
tree: ast.Module = ast.parse(code)
function: ast.FunctionDef = tree.body[0]

print(ast.get_docstring(function))

Args:
    param: Literally anything!

Return:
    Literally anything!


**Спойлер.** На семинаре:

- Изучите дополнительные примеры работы с абстрактными синтаксическими деревьями.